# Bayesian Exploration
Here we demonstrate the use of Bayesian Exploration to characterize an unknown
function in the presence of constraints (see [here](https://www.nature.com/articles/s41467-021-25757-3)).
The function we wish to explore is the first objective of the TNK test problem.

## Specifiying generator options
We start with the generator defaults and modify as needed for conservative
exploration, which should prevent any constraint violations.

In [ ]:
# set values if testing
import os

from copy import deepcopy
from xopt import Xopt, Evaluator
from xopt.generators.bayesian import BayesianExplorationGenerator
from xopt.resources.test_functions.tnk import evaluate_TNK, tnk_vocs

# Ignore all warnings
import warnings

warnings.filterwarnings("ignore")

SMOKE_TEST = os.environ.get("SMOKE_TEST")
NUM_MC_SAMPLES = 1 if SMOKE_TEST else 128
NUM_RESTARTS = 1 if SMOKE_TEST else 20


vocs = deepcopy(tnk_vocs)
vocs.objectives = {}
vocs.observables = ["y1"]

generator = BayesianExplorationGenerator(vocs=vocs)
generator.numerical_optimizer.n_restarts = NUM_RESTARTS
generator.numerical_optimizer.max_iter = 100
generator.n_monte_carlo_samples = NUM_MC_SAMPLES
generator.n_interpolate_points = 5

evaluator = Evaluator(function=evaluate_TNK)

X = Xopt(generator=generator, evaluator=evaluator, vocs=vocs)
X

## Run exploration
We start with evaluating 2 points that we know satisfy the constraints. We then run
30 exploration steps.

In [ ]:
X.evaluate_data({"x1": [1.0, 0.75], "x2": [0.7, 0.95]})

In [ ]:
for i in range(20):
    print(f"step {i}")
    X.step()

In [ ]:
# view the data
X.data

In [ ]:
# plot results
ax = X.data.plot("x1", "x2")
ax.set_aspect("equal")

## Introspect models, acquisition function and feasibility prediction
During exploration we generate Gaussian Process models of each objective and
constraint. We demonstrate how they are viewed below.

In [ ]:
fig, ax = X.generator.visualize_model(show_feasibility=True, n_grid=100)

## Generator model hyperparameters

In [ ]:
# print generator model hyperparameters
for name, val in X.generator.model.named_parameters():
    print(f"{name}:{val}")

X.generator.model.models[2].covar_module.base_kernel.lengthscale

## Examine the number of constraint violations
Using the convience function provided by the vocs object we can evaluate which
samples violate either or both of our constraints.

In [ ]:
X.vocs.feasibility_data(X.data)

In [ ]:
# generate next point
X.generator.generate(1)